In [2]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
import datetime as dt
import logging
import optuna


import zipfile


from mlxtend.evaluate.time_series import (
    GroupTimeSeriesSplit,
    plot_splits,
    print_cv_info,
    print_split_info,
)


# Scikit-learn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.preprocessing import RobustScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer


from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.base import clone, BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectPercentile, mutual_info_regression

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.ensemble import StackingRegressor, RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_absolute_error, PredictionErrorDisplay
# Gradient Boosting Models
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import xgboost as xgb

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline
import gc
import torch
# Bayesian Optimization
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# Kaggle API (only import if not running in a Kaggle notebook)
iskaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "")
if not iskaggle:
    import kaggle
    from kaggle.api.kaggle_api_extended import KaggleApi

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

c:\Users\Lawrence\Documents\PYTHON\1_kaggle_competitions\rohlik_feb_25\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Clear CPU memory
gc.collect()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [4]:
class RohlikDataProcessor:
    def __init__(self, is_kaggle, path="rohlik-sales-forecasting-challenge-v2"):
        self.path = path
        self.column_transformer = None
        self.is_kaggle = is_kaggle

    def download_and_extract(self):
        if not self.is_kaggle and not Path(self.path).exists():
            zip_path = Path(f"{self.path}.zip")
            if not zip_path.exists():
                import kaggle
                kaggle.api.competition_download_cli('rohlik-sales-forecasting-challenge-v2')
            with zipfile.ZipFile(zip_path, 'r') as zf:
                zf.extractall(self.path)
            print("Dataset extracted successfully.")
        
        if self.is_kaggle:
            self.path = '/kaggle/input/rohlik-sales-forecasting-challenge-v2'  # Set Kaggle path explicitly

    def load_data(self, filename):
        return pd.read_csv(f"{self.path}/{filename}")
        
    def add_holidays_to_calender_df(self, calendar_raw_df):
        #taken from https://www.kaggle.com/code/macarrony00/not-a-winner-but-maybe-some-inspiration
        czech_holiday = [ 
            (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
            (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
        ]
        brno_holiday = [
            (['03/31/2024', '04/09/2023', '04/17/2022', '04/04/2021', '04/12/2020'], 'Easter Day'),#loss
            (['05/12/2024', '05/10/2020', '05/09/2021', '05/08/2022', '05/14/2023'], "Mother Day"), #loss
        ]
        
        budapest_holidays = []
        munich_holidays = [
            (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
            (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
        ]
        
        frank_holidays = [
            (['03/30/2024', '04/08/2023', '04/16/2022', '04/03/2021'], 'Holy Saturday'),#loss
            (['05/12/2024', '05/14/2023', '05/08/2022', '05/09/2021'], 'Mother Day'),#loss
        ]
    
        def fill_loss_holidays(df_fill, warehouses, holidays):
            df = df_fill.copy()
            for item in holidays:
                dates, holiday_name = item
                generated_dates = [datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d') for date in dates]
                for generated_date in generated_dates:
                    df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday'] = 1
                    df.loc[(df['warehouse'].isin(warehouses)) & (df['date'] == generated_date), 'holiday_name'] = holiday_name
            return df
        
        calendar_with_holidays_df = fill_loss_holidays(df_fill=calendar_raw_df, warehouses=['Prague_1', 'Prague_2', 'Prague_3'], holidays=czech_holiday)
        calendar_with_holidays_df = fill_loss_holidays(df_fill=calendar_with_holidays_df, warehouses=['Brno_1'], holidays=brno_holiday)
        calendar_with_holidays_df = fill_loss_holidays(df_fill=calendar_with_holidays_df, warehouses=['Munich_1'], holidays=munich_holidays)
        calendar_with_holidays_df = fill_loss_holidays(df_fill=calendar_with_holidays_df, warehouses=['Frankfurt_1'], holidays=frank_holidays)
        calendar_with_holidays_df = fill_loss_holidays(df_fill=calendar_with_holidays_df, warehouses=['Budapest_1'], holidays=budapest_holidays)
        return calendar_with_holidays_df
    

    def add_calender_and_inventory(self, df, calendar_with_holidays_df, inventory_raw_df):
        df_with_inventory = df.merge(inventory_raw_df[['unique_id', 'product_unique_id', 'name', 
                                      'L1_category_name_en', 'L2_category_name_en', 
                                      'L3_category_name_en', 'L4_category_name_en']], 
                        on='unique_id', how='left')
        
        df_with_inventory_and_calandar = df_with_inventory.merge(calendar_with_holidays_df[['date', 'warehouse', 'holiday', 'holiday_name', 
                                     'shops_closed', 'winter_school_holidays', 'school_holidays']], 
                        on=['date', 'warehouse'], how='left')
        return df_with_inventory_and_calandar


    def preprocess(self, train_df, test_df, target_col):

        train_df['sales'] = train_df['sales'].fillna(0)
        train_df['total_orders'] = train_df['total_orders'].fillna(0)
        train_df['sell_price_main'] = train_df['sell_price_main'].interpolate()

        train_df.drop(columns=['availability'], inplace=True)

        train_df['date'] = pd.to_datetime(train_df['date'])
        test_df['date'] = pd.to_datetime(test_df['date'])
        
        train_df['is_test'] = 0
        test_df['is_test'] = 1

        df = pd.concat([train_df, test_df], axis=0)
        
        
        discount_columns = ['type_0_discount', 'type_1_discount', 'type_2_discount', 
                            'type_3_discount', 'type_4_discount', 'type_5_discount', 'type_6_discount']
        df['max_discount'] = df[discount_columns].max(axis=1)
        
        
        
        df = df.sort_values(by=['product_unique_id', 'warehouse', 'date'])
        
        periods = [3,7,10,14]
        for p in periods: 
            
            df['total_orders_mean_' + str(p)] = df.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
            .rolling(window=p, min_periods=1) \
            .mean() \
            .reset_index(level=[0, 1], drop=True)
        
            df['total_orders_std_' + str(p)] = df.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
            .rolling(window=p, min_periods=1) \
            .std() \
            .reset_index(level=[0, 1], drop=True)
        
            df['total_orders_max_' + str(p)] = df.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
            .rolling(window=p, min_periods=1) \
            .max() \
            .reset_index(level=[0, 1], drop=True)
        
            df['total_orders_min_' + str(p)] = df.groupby(['product_unique_id', 'warehouse'])['total_orders'] \
            .rolling(window=p, min_periods=1) \
            .min() \
            .reset_index(level=[0, 1], drop=True)
        
        for lag in range(1, 15):  
            df[f'lag_{lag}'] = df.groupby(['product_unique_id', 'warehouse'])['sales'].shift(lag)
            df[f'lag_{lag}'] = df[f'lag_{lag}'].fillna(df.groupby(['product_unique_id', 'warehouse'])['sales'].transform("last"))
        
        
        categorical_columns = ['warehouse', 'name', 'L1_category_name_en', 'L2_category_name_en', 
                               'L3_category_name_en', 'L4_category_name_en', 'holiday_name']
        
            
        for col in categorical_columns:
            df[col] = df[col].astype('category')

        return df

    def add_date_columns(self, df):
        
        df["datetime"] = pd.to_datetime(df["date"])
        df["month"] = df["datetime"].dt.month
        df["day"] = df["datetime"].dt.day
        df["weekday"] = df["datetime"].dt.weekday
        df["quarter"] = df["datetime"].dt.quarter
        df["week_of_year"] = df["datetime"].dt.isocalendar().week
        df["day_of_year"] = df["datetime"].dt.dayofyear
        df["is_weekend"] = df["datetime"].dt.weekday.isin([5, 6]).astype(int)
        df["is_month_start"] = df["datetime"].dt.is_month_start.astype(int)
        df["is_month_end"] = df["datetime"].dt.is_month_end.astype(int)
        df['year_sin'] = np.sin(2 * np.pi * df['datetime'].dt.year)
        df['year_cos'] = np.cos(2 * np.pi * df['datetime'].dt.year)
        df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12) 
        df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
        df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)  
        df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)
        df_with_dates = df.copy()
        
        return df_with_dates
    
    def split_by_warehouse(self, df):
        df_warehouse_list = []
        for warehouse in df['warehouse'].unique():
            warehouse_df = df[df['warehouse'] == warehouse]
            df_warehouse_list.append(warehouse_df)
        
        return df_warehouse_list

In [5]:
processor = RohlikDataProcessor(iskaggle, path="rohlik-sales-forecasting-challenge-v2")
processor.download_and_extract()

In [6]:
sales_train_raw_df = processor.load_data("sales_train.csv")
sales_test_raw_df = processor.load_data("sales_test.csv")
calendar_raw_df = processor.load_data("calendar.csv")
inventory_raw_df = processor.load_data("inventory.csv")
solution_raw_df = processor.load_data("solution.csv")
test_weights_raw_df = processor.load_data("test_weights.csv")

In [7]:
calendar_with_holidays_df = processor.add_holidays_to_calender_df(calendar_raw_df)

In [8]:
sales_train_df_with_inventory_and_calandar = processor.add_calender_and_inventory(sales_train_raw_df, calendar_with_holidays_df, inventory_raw_df)
sales_test_df_with_inventory_and_calandar = processor.add_calender_and_inventory(sales_test_raw_df, calendar_with_holidays_df, inventory_raw_df)

In [9]:
full_df = processor.preprocess(sales_train_df_with_inventory_and_calandar, sales_test_df_with_inventory_and_calandar, 'Sales')

In [10]:
full_with_dates_df = processor.add_date_columns(full_df)

In [11]:
weight_map = test_weights_raw_df.set_index('unique_id')['weight'].to_dict()

In [12]:
full_with_dates_df.columns

Index(['unique_id', 'date', 'warehouse', 'total_orders', 'sales',
       'sell_price_main', 'type_0_discount', 'type_1_discount',
       'type_2_discount', 'type_3_discount', 'type_4_discount',
       'type_5_discount', 'type_6_discount', 'product_unique_id', 'name',
       'L1_category_name_en', 'L2_category_name_en', 'L3_category_name_en',
       'L4_category_name_en', 'holiday', 'holiday_name', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'is_test', 'max_discount',
       'total_orders_mean_3', 'total_orders_std_3', 'total_orders_max_3',
       'total_orders_min_3', 'total_orders_mean_7', 'total_orders_std_7',
       'total_orders_max_7', 'total_orders_min_7', 'total_orders_mean_10',
       'total_orders_std_10', 'total_orders_max_10', 'total_orders_min_10',
       'total_orders_mean_14', 'total_orders_std_14', 'total_orders_max_14',
       'total_orders_min_14', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5',
       'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 

In [13]:
warehouse_df_list = processor.split_by_warehouse(full_with_dates_df)
print(len(warehouse_df_list))

7


In [14]:
target = 'sales'

# Define columns for different encoding strategies
passthrough_columns = ["unique_id", "is_test", "sample_weight"]
one_hot_cols = ["L1_category_name_en", "L2_category_name_en"]
label_encode_cols = list(set(full_with_dates_df.select_dtypes(include="category").columns) - set(one_hot_cols))
numeric_columns = list(full_with_dates_df.select_dtypes(exclude="category").columns.difference(passthrough_columns))

# Numeric transformer
numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")), 
    ("scaler", RobustScaler())
])

# One-hot encoding transformer (for low-cardinality categorical features)
one_hot_transformer = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# Label encoding transformer (for high-cardinality categorical features)
label_encoder_transformer = Pipeline([
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])

# Combine transformations using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("onehot", one_hot_transformer, one_hot_cols),
        ("label", label_encoder_transformer, label_encode_cols),
    ],
    remainder='passthrough',
    n_jobs=-1
)

# Define models
reg1 = Pipeline([
    ("preprocessor", preprocessor), 
    ("regressor", lgb.LGBMRegressor(
        verbosity=-1, 
        device="gpu"
    ))
])

features = ["unique_id",
    "warehouse", "total_orders", 
    "sell_price_main", "type_0_discount", "type_1_discount", "type_2_discount", 
    "type_3_discount", "type_4_discount", "type_5_discount", "type_6_discount", 
    "name", "L1_category_name_en", "L2_category_name_en", 
    "L3_category_name_en", "L4_category_name_en", "holiday", "holiday_name", 
    "shops_closed", "winter_school_holidays", "school_holidays", 
    "max_discount", "total_orders_mean_3", "total_orders_std_3", "total_orders_max_3", 
    "total_orders_min_3", "total_orders_mean_7", "total_orders_std_7", "total_orders_max_7", 
    "total_orders_min_7", "total_orders_mean_10", "total_orders_std_10", "total_orders_max_10", 
    "total_orders_min_10", "total_orders_mean_14", "total_orders_std_14", "total_orders_max_14", 
    "total_orders_min_14", "month", "day", "weekday", "quarter", "week_of_year", 
    "day_of_year", "is_weekend", "is_month_start", "is_month_end", "year_sin", 
    "year_cos", "month_sin", "month_cos", "day_sin", "day_cos",
    "lag_1", "lag_2", "lag_3", "lag_4", "lag_5", "lag_6", "lag_7", 
    "lag_8", "lag_9", "lag_10", "lag_11", "lag_12", "lag_13", "lag_14"
]


In [15]:
print(reg1)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000025918AEAED0>),
                                                 ('onehot',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
      

In [ ]:
# Initialize dictionaries to store results
best_params_dict = {}
best_score_dict = {}
cv_results_dict = {}
test_predictions_dict = {}

compute_test = True  # Set this to True to compute test predictions

# Define cross-validation settings
cv_args = {'n_splits': 5, 'test_size': 14}
gtscv = TimeSeriesSplit(**cv_args)

def weighted_mae(y_true, y_pred, sample_weight):
    return np.sum(sample_weight * np.abs(y_true - y_pred)) / np.sum(sample_weight)

weighted_mae_scorer = make_scorer(weighted_mae, greater_is_better=False)

# Define Optuna objective function
def objective(trial):
    params = {
        'regressor__learning_rate': trial.suggest_float('regressor__learning_rate', 0.01, 0.1, log=True),
        'regressor__n_estimators': trial.suggest_int('regressor__n_estimators', 100, 10000),
        'regressor__num_leaves': trial.suggest_int('regressor__num_leaves', 20, 150),
        'regressor__boosting_type': trial.suggest_categorical('regressor__boosting_type', ['gbdt', 'dart']),
        'regressor__max_depth': trial.suggest_int('regressor__max_depth', 3, 15),
        'regressor__min_child_samples': trial.suggest_int('regressor__min_child_samples', 10, 50),
        'regressor__max_bin': trial.suggest_int('regressor__max_bin', 100, 255),
        'regressor__colsample_bytree': trial.suggest_float('regressor__colsample_bytree', 0.5, 1.0),
        'regressor__subsample': trial.suggest_float('regressor__subsample', 0.5, 1.0),
        'regressor__reg_alpha': trial.suggest_float('regressor__reg_alpha', 1e-4, 1.0, log=True),
        'regressor__reg_lambda': trial.suggest_float('regressor__reg_lambda', 1e-4, 1.0, log=True),
        'regressor__device': 'gpu'
    }

    reg1.set_params(**params)  # Update the pipeline with new parameters

    scores = []
    for warehouse in warehouse_df_list:
        df = warehouse.sort_values(by=['date']).set_index('date').copy()
        df_train = df[df['is_test'] == 0]
        if df_train.empty:
            continue

        X = df_train[features].copy()
        X["sample_weight"] = X["unique_id"].map(weight_map).fillna(1)
        y = df_train[target] ** (1/8)  # 1/8th root transformation

        if X.empty or y.empty:
            continue

        for train_idx, test_idx in gtscv.split(X):
            X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

            reg1.fit(X_train, y_train, regressor__sample_weight=X_train["sample_weight"])
            y_pred = reg1.predict(X_val)
            score = weighted_mae(y_val, y_pred, X_val["sample_weight"])
            scores.append(score)4

            # Prune unpromising trials
            trial.report(np.mean(scores), len(scores))
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

    return np.mean(scores)

# Run Optuna optimization
study = optuna.create_study(direction='minimize', pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=3)

# Apply the best parameters to each warehouse
for warehouse in warehouse_df_list:
    df = warehouse.sort_values(by=['date']).set_index('date').copy()

    df_train = df[df['is_test'] == 0]
    df_test = df[df['is_test'] == 1]

    print(f"Processing warehouse {df['warehouse'].iloc[0]}")
    if df_train.empty:
        print(f"Skipping warehouse {df['warehouse'].iloc[0]} due to empty training data.")
        continue

    X = df_train[features].copy()
    X["sample_weight"] = X["unique_id"].map(weight_map).fillna(1)
    y = df_train[target] ** (1/8)  # 1/8th root transformation

    if X.empty or y.empty:
        print(f"Skipping warehouse {df['warehouse'].iloc[0]} due to empty feature set.")
        continue

    if X["sample_weight"].isna().sum() > 0:
        print(f"Warning: NaN values found in sample weights for warehouse {df['warehouse'].iloc[0]}")

    # Store results
    warehouse_name = df['warehouse'].iloc[0]
    best_params_dict[warehouse_name] = study.best_params
    best_score_dict[warehouse_name] = study.best_value
    cv_results_dict[warehouse_name] = study.trials_dataframe()

    if compute_test:
        # Use the best estimator to make predictions on df_test
        best_estimator = reg1.set_params(**study.best_params)
        X_test = df_test[features].copy()
        X_test["sample_weight"] = X_test["unique_id"].map(weight_map).fillna(1)
        y_test_pred = best_estimator.predict(X_test)
        test_predictions_dict[warehouse_name] = y_test_pred
        # Clear CPU memory
        gc.collect()

        # Clear GPU memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

# Print the results
print("Best Parameters:", best_params_dict)
print("Best Scores:", best_score_dict)
if compute_test:
    print("Test Predictions:", test_predictions_dict)


[I 2025-02-25 17:42:54,392] A new study created in memory with name: no-name-441b9fdc-b04a-40b5-97d0-789aa646a6b4
c:\Users\Lawrence\Documents\PYTHON\1_kaggle_competitions\rohlik_feb_25\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Lawrence\Documents\PYTHON\1_kaggle_competitions\rohlik_feb_25\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Lawrence\Documents\PYTHON\1_kaggle_competitions\rohlik_feb_25\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Lawrence\Documents\PYTHON\1_kaggle_competitions\rohlik_feb_25\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarni

In [ ]:
# if COMPUTE_TEST:
#     submission_df = sales_test.copy()
#     submission_df['sales_hat'] = test_preds['StackingRegressor'].clip(0,None)
#     submission_df['id'] = submission_df['unique_id'].astype(str) + "_" + submission_df['date'].dt.strftime('%Y-%m-%d')
#     submission_df[['id', 'sales_hat']].to_csv('submission.csv', index=False)